In [1]:
from ast import literal_eval
from collections import defaultdict

import pandas as pd

import wandb
from scripts.results_processing.preprocess import preprocess_df
from scripts.results_processing.reproduction.constants import (
    dataset_model_columns,
    optimization_metrics,
    run_columns,
    sweeped_columns,
)

%load_ext autoreload
%autoreload 2


In [2]:

columns_to_normalize = [
    "model",
    "dataset",
    "transforms",
    "optimizer",
    "callbacks",
]
def normalize_columns(df, columns_to_normalize):
    # Gather the new DataFrames to be concatenated
    flattened_dfs = []

    for col in columns_to_normalize:
        df[col] = df[col].apply(lambda x: str(x).replace("nan", "None"))
        df[col] = df[col].apply(literal_eval)

        # Flatten
        flat = pd.json_normalize(df[col])

        # Rename
        flat.columns = [f"{col}.{c}" for c in flat.columns]
        flattened_dfs.append(flat)

    # Drop all nested columns in one shot
    df = df.drop(columns=columns_to_normalize)

    # Concatenate once at the end
    return pd.concat([df] + flattened_dfs, axis=1)


def normalize_df(df, columns_to_normalize):
    # Config columns to normalize
    df = normalize_columns(df, columns_to_normalize)

    return df

def fetch(project):
    user = "telyatnikov_sap"
    api = wandb.Api(overrides={"base_url": "https://api.wandb.ai"}, timeout=40)
    runs = api.runs(f"{user}/{project}")
    summary_list, config_list, name_list = [], [], []
    for run in runs:
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        config_list.append(
            {
                k: v
                for k, v in run.config.items()
                if not k.startswith("_")
            }
        )

        # .name is the human-readable name of the run.
        name_list.append(run.name)

    runs_df = pd.DataFrame(
        {
            "summary": summary_list,
            "config": config_list,
            "name": name_list,
        }
    )
    # Merge the dicts in a vectorized way:
    merged_dicts = [
        {**s, **c}
        for s, c in zip(runs_df["summary"], runs_df["config"], strict=False)
    ]

    # Now expand them into a DataFrame:
    df_merged = pd.DataFrame.from_records(merged_dicts)
    return df_merged

def main():
    df_list = [ ]
    projects = ["CSL", "CSL_sann", "CSL_sccnn", "Topotune_CSL", "Topotune_cell_CSL"]
    for project in projects:
        tmp_df = fetch(project)
        df_list.append(tmp_df)
    df = pd.concat(df_list, ignore_index=True)
    df = normalize_df(df, columns_to_normalize)
    return df

In [3]:
def gen_scores(df):
    # Get unique datasets
    datasets = list(df["dataset.loader.parameters.data_name"].unique())
    # Get unique models
    models = list(df["model.model_name"].unique())

    collect_subsets = defaultdict(dict)
    # Got over each dataset and model and find the best result
    for dataset in datasets:
        for model in models:
            # Get the subset of the DataFrame for the current dataset and model
            subset = df.loc[
                (df["dataset.loader.parameters.data_name"] == dataset)
            ]

            optim_metric = optimization_metrics[dataset]["optim_metric"]
            eval_metric = optimization_metrics[dataset]["eval_metric"]
            direction = optimization_metrics[dataset]["direction"]

            # Keep metrics that matters for dataset
            performance_columns = optimization_metrics[dataset][
                "performance_columns"
            ]
            subset = subset[
                dataset_model_columns
                + sweeped_columns
                + performance_columns
                + run_columns
            ]
            aggregated = subset.groupby(
                sweeped_columns + ["model.model_name", "model.model_domain"],
                dropna=False,
            ).agg(
                {col: ["mean", "std", "count", "max", "min"] for col in performance_columns},
            )

            # aggregated = subset.groupby(sweeped_columns, dropna=False).count()

            n_count = 5 if "MANTRA" not in dataset else 4
            # Go from MultiIndex to Index
            aggregated = aggregated.reset_index()
            print(f"Dataset: {dataset}, Model: {model}")
            print(aggregated[(eval_metric, "count")].unique())
            # print(aggregated['dataset.split_params.data_seed'].unique())
            print(
                (aggregated[(eval_metric, "count")] >= n_count).sum()
                / len(aggregated)
                * 100
            )
            aggregated = aggregated[aggregated[(eval_metric, "count")] >= n_count]
            # print(len(aggregated[aggregated['seed'] > 4]))
            # aggregated = aggregated.sort_values(
            #     by=(optim_metric, "mean"), ascending=(direction == "min")
            # )

            # Git percent in case of classification
            if "test/accuracy" in performance_columns:
                # Go over all the performance columns and multiply by 100
                for col in performance_columns:
                    aggregated[(col, "mean")] *= 100
                    aggregated[(col, "std")] *= 100
                    aggregated[(col, "max")] *= 100
                    aggregated[(col, "min")] *= 100

                # Round performance columns values up to 2 decimal points
                for col in performance_columns:
                    aggregated[(col, "mean")] = aggregated[
                        (col, "mean")
                    ].round(2)
                    aggregated[(col, "max")] = aggregated[(col, "max")].round(
                        2
                    )
                    aggregated[(col, "min")] = aggregated[(col, "min")].round(
                        2
                    )

            else:
                # Round all values up to 4 decimal points
                # Round performance columns values up to 4 decimal points
                for col in performance_columns:
                    aggregated[(col, "mean")] = aggregated[
                        (col, "mean")
                    ].round(4)
                    aggregated[(col, "std")] = aggregated[(col, "std")].round(
                        4
                    )

            collect_subsets[dataset] = aggregated
    return collect_subsets


In [4]:
df = main()

In [5]:
df = preprocess_df(df, split_mantra=False)

In [6]:
collected_subsets = gen_scores(df)

Dataset: CSL, Model: hopse_g
[ 5  4 88 89 90]
98.94736842105263
Dataset: CSL, Model: HOPSE_MANUAL_PE
[ 5  4 88 89 90]
98.94736842105263
Dataset: CSL, Model: SANN
[ 5  4 88 89 90]
98.94736842105263
Dataset: CSL, Model: sccnn
[ 5  4 88 89 90]
98.94736842105263
Dataset: CSL, Model: topotune
[ 5  4 88 89 90]
98.94736842105263


In [7]:
collected_subsets["CSL"].sort_values(by=("val/auroc", "mean"), ascending=False)[["model.model_name", "val/auroc", "test/auroc", "model.backbone.n_layers", "dataset.dataloader_params.batch_size", "optimizer.parameters.lr", "optimizer.parameters.weight_decay", "transforms.sann_encoding.pretrain_model", "transforms.sann_encoding.neighborhoods"]].head(60)

model.model_name val/auroc                                  test/auroc  \
                          mean        std count     max     min       mean   
290  HOPSE_MANUAL_PE    100.00   0.000000     5  100.00  100.00     100.00   
96           hopse_g     99.60   0.903028     5  100.00   97.98      99.60   
291  HOPSE_MANUAL_PE     93.14   4.121533     5   96.62   86.89      91.94   
372         topotune     83.48  17.641148    90  100.00   49.75      79.73   
376         topotune     83.22  18.571034    90  100.00   40.17      79.07   
364         topotune     82.57  19.714335    88  100.00   39.85      80.34   
368         topotune     82.40  20.472845    90  100.00   41.45      80.31   
370         topotune     75.56  21.273444    89  100.00   39.18      72.56   
366         topotune     74.65  21.126191    90  100.00   38.67      71.96   
378         topotune     70.55  22.366105    90  100.00   37.20      68.53   
374         topotune     68.60  21.665445    90  100.00   32.99      66.12   
353            sccnn     65.52   5.948770     5   72.34   59.46      55.03   
130             SANN     65.23   4.122569     5   70.10   60.87      60.87   
114             SANN     64.59   5.441130     5   71.80   57.93      60.06   
262             SANN     64.31   4.555437     5   71.61   60.43      60.42   
266             SANN     64.18   5.701033     5   71.42   56.04      61.53   
279             SANN     64.12   7.494982     5   73.04   55.77      61.56   
123             SANN     64.08   5.507231     5   70.95   55.57      59.05   
283             SANN     64.00   5.405448     5   70.14   57.40      60.61   
97           hopse_g     63.94   5.517961     5   69.71   56.29      61.77   
231             SANN     63.93  10.676646     5   74.27   47.29      59.88   
146             SANN     63.79   6.500964     5   72.46   56.66      61.60   
156             SANN     63.68   4.589862     5   70.46   58.51      61.14   
274             SANN     63.49   6.636300     5   71.58   57.02      64.91   
215             SANN     63.38   6.617933     5   69.28   56.22      63.72   
293            sccnn     63.34   9.274651     5   76.71   53.76      55.92   
163             SANN     63.30   6.377498     5   70.54   58.20      60.30   
280             SANN     63.13   5.059131     5   70.52   57.82      63.54   
205             SANN     63.10   7.102136     5   71.51   56.19      56.61   
166             SANN     63.00   5.226933     5   66.95   54.34      59.22   
271             SANN     62.99   2.729680     5   66.83   60.16      61.34   
287             SANN     62.97   3.406904     5   65.91   57.49      60.96   
145             SANN     62.95   3.591451     5   68.40   59.66      62.07   
229             SANN     62.93   5.877717     5   69.81   54.24      60.70   
184             SANN     62.90   1.683950     5   65.54   61.17      61.17   
110             SANN     62.89   4.675418     5   67.33   55.92      60.98   
246             SANN     62.87   4.885308     5   67.91   55.78      59.58   
128             SANN     62.84   3.910269     5   66.87   57.97      59.11   
254             SANN     62.80   4.073381     5   66.07   56.26      59.61   
171             SANN     62.66   7.645694     5   71.41   53.55      61.97   
304            sccnn     62.63   7.350016     5   71.10   54.19      61.31   
129             SANN     62.56   1.939947     5   65.76   60.46      60.09   
153             SANN     62.44   5.001157     5   68.32   56.69      59.05   
170             SANN     62.33   8.458697     5   76.75   54.97      62.49   
102             SANN     62.31   7.315598     5   69.28   54.29      59.55   
256             SANN     62.30   7.545484     5   68.70   49.49      62.46   
325            sccnn     62.26  13.991692     5   74.88   40.18      59.79   
107             SANN     62.10   5.381024     5   68.45   55.86      58.88   
242             SANN     62.06   6.286310     5   67.55   53.60      58.98   
182             SANN     6

In [8]:
models = list(df["model.model_name"].unique())
domains = list(df["model.model_domain"].unique())

df_dict = {
    "model": [],
    "domain": [],
    "mean": [],
    "std": [],
    "max": [],
    "min": []
}

for model in models:
    for domain in domains:
        subset = collected_subsets["CSL"][
            (collected_subsets["CSL"]["model.model_name"] == model)
            & (collected_subsets["CSL"]["model.model_domain"] == domain)
        ]
        if subset.empty:
            continue
        optim_metric = optimization_metrics["CSL"]["optim_metric"]
        eval_metric = optimization_metrics["CSL"]["eval_metric"]
        direction = optimization_metrics["CSL"]["direction"]

        subset = subset.sort_values(by=(optim_metric, "mean"), ascending=(direction == "min"))
        best_result = subset.iloc[0]

        df_dict["model"].append(model)
        df_dict["domain"].append(domain)
        df_dict["mean"].append(best_result[(eval_metric, "mean")])
        df_dict["std"].append(best_result[(eval_metric, "std")])
        df_dict["max"].append(best_result[(eval_metric, "max")])
        df_dict["min"].append(best_result[(eval_metric, "min")])
df_res = pd.DataFrame(df_dict)
         


In [9]:
df_res

,model,domain,mean,std,max,min
0,hopse_g,cell,98.00,4.472137,100.00,90.00
1,hopse_g,simplicial,18.67,6.912147,26.67,10.00
2,HOPSE_MANUAL_PE,cell,100.00,0.000000,100.00,100.00
3,HOPSE_MANUAL_PE,simplicial,60.00,11.303884,73.33,46.67
4,SANN,simplicial,26.00,6.831301,33.33,20.00
5,sccnn,simplicial,25.33,7.673910,33.33,16.67
6,topotune,cell,52.78,34.286921,100.00,3.33
7,topotune,simplicial,24.16,5.801448,33.33,13.33
